In [63]:
import numpy as np
from scipy import interpolate
from scipy.signal import find_peaks
from matplotlib import pyplot as plt
from analyse_contacts_ref import read_expe1, read_expe2, read_expe3, ground_truth
import sklearn
%matplotlib notebook

# read datasets

In [112]:
t_arr1, mocapd1, measd1 = read_expe1()
t_arr2, mocapd2, measd2 = read_expe2()
t_arr3, mocapd3, measd3 = read_expe3()
N1 = len(t_arr1)
N2 = len(t_arr2)
N3 = len(t_arr3)

In [65]:
# features in 

# create ground truth

In [66]:
# to get ground truth :
# - foot height
# - foot velocity

# to obtain:  
# height with respect to previous lowest height

In [67]:
y1 = ground_truth(mocapd1['FL'][:,2])
y2 = ground_truth(mocapd2['FL'][:,2])
y3 = ground_truth(mocapd3['FL'][:,2])

# KNN

In [68]:
from sklearn.neighbors import KNeighborsClassifier

In [69]:
# data
X1 = np.zeros((len(t_arr1), 3))
X1[:, 0:3] = measd1['tau'][:, 0:3]  # FL .. .. ..

X2 = np.zeros((len(t_arr1), 3))
X2[:, 0:3] = measd1['tau'][:, 0:3]  # FL .. .. ..

In [70]:
clf = KNeighborsClassifier(n_neighbors=2)

In [71]:
clf.fit(X1, y1)

KNeighborsClassifier(n_neighbors=2)

In [72]:
clf.predict(X2)

array([0, 0, 0, ..., 0, 0, 0])

# SVC

In [84]:
from sklearn.model_selection import cross_val_score

In [85]:
C = 0.01 # SVM regularization parameter
n_components = 2
estimators = [('scaler', sklearn.preprocessing.StandardScaler()),
              ('reduce_dim', sklearn.decomposition.IncrementalPCA(n_components=n_components, batch_size=100)),
              ('clf', sklearn.svm.SVC(kernel='linear', C=C))]
pca_svc_clf = sklearn.pipeline.Pipeline(estimators)

estimators = [('scaler', sklearn.preprocessing.StandardScaler()),
              ('clf', sklearn.svm.SVC(kernel='linear', C=C))]
svc_clf = sklearn.pipeline.Pipeline(estimators)

In [125]:
X1 = np.zeros((len(t_arr1), 6))
X1[:, 0:3] = measd1['tau'][:, 0:3]  # FL .. .. ..
X1[:, 0:3] = measd1['qa'][:, 0:3]  # FL .. .. ..

X2 = np.zeros((len(t_arr2), 6))
X2[:, 0:3] = measd1['tau'][:, 0:3]  # FL .. .. ..
X2[:, 0:3] = measd1['qa'][:, 0:3]  # FL .. .. ..

X3 = np.zeros((len(t_arr3), 6))
X3[:, 0:3] = measd1['tau'][:, 0:3]  # FL .. .. ..
X3[:, 0:3] = measd1['qa'][:, 0:3]  # FL .. .. ..



In [126]:
scores_pca_svc = cross_val_score(pca_svc_clf, X1, y1, cv=5)
scores_svc = cross_val_score(svc_clf, X1, y1, cv=5)

In [124]:
print(scores_pca_svc.mean(), scores_pca_svc.std())
print(scores_svc.mean(), scores_svc.std())

0.8711666666666666 0.006176793486735471
0.9008333333333333 0.004533823502911809


In [127]:
print(scores_pca_svc.mean(), scores_pca_svc.std())
print(scores_svc.mean(), scores_svc.std())

0.9731666666666665 0.004390646624106486
0.9753333333333332 0.004119735698102746


In [105]:
pca_svc_clf.fit(X1, y1)
svc_clf.fit(X1, y1)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', SVC(C=0.01, kernel='linear'))])

In [106]:
# Test sets
y2_pred_pca_svc = pca_svc_clf.predict(X2)
y3_pred_pca_svc = pca_svc_clf.predict(X3)

y2_pred_svc = svc_clf.predict(X2)
y3_pred_svc = svc_clf.predict(X3)

In [110]:
print(sklearn.metrics.f1_score(y2_pred_pca_svc, y2))
print(sklearn.metrics.f1_score(y3_pred_pca_svc, y3))
print(sklearn.metrics.f1_score(y2_pred_svc, y2))
print(sklearn.metrics.f1_score(y3_pred_svc, y3))

0.9453809844908968
0.9339601522274457
0.9445442299057027
0.9306797853309481


In [113]:
print(sklearn.metrics.confusion_matrix(y2_pred_pca_svc, y2)*100/N2)
print(sklearn.metrics.confusion_matrix(y3_pred_pca_svc, y3)*100/N3)
print(sklearn.metrics.confusion_matrix(y2_pred_svc, y2)*100/N2)
print(sklearn.metrics.confusion_matrix(y3_pred_svc, y3)*100/N3)

[[60.9    3.575]
 [ 0.475 35.05 ]]
[[60.31666667  4.15833333]
 [ 0.75833333 34.76666667]]
[[60.825       3.56666667]
 [ 0.55       35.05833333]]
[[60.15        4.24166667]
 [ 0.925      34.68333333]]


In [121]:
%%timeit
pca_svc_clf.predict(X2[2,:].reshape((1,6)))

194 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
